In [383]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
import plotly.express as px

import category_encoders as ce

from sklearn import preprocessing

import geopy.geocoders
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")



In [5]:
pip install category_encoders


   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/82.0 kB ? eta -:--:--
   -------------- ------------------------- 30.7/82.0 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 82.0/82.0 kB 658.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install geopy


   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [329]:
dtrain = pd.read_csv('train.revenue.csv')

In [331]:
dtrain.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [332]:
dtrain.shape

(137, 43)

In [333]:
dtest = pd.read_csv('test.revenue.csv')

In [334]:
dtest.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,2.0,3.0,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,1.0,3.0,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,2.0,3.0,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,2.0,3.0,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,5.0,3.0,0,0,0,0,0,0,0,0


In [335]:
dtest.shape

(100000, 42)

In [336]:
dtest.isna().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
dtype: int64

In [337]:
dtrain.describe().T.style.background_gradient(cmap = 'magma',axis = 1)

,count,mean,std,min,25%,50%,75%,max
Id,137.000000,68.000000,39.692569,0.000000,34.000000,68.000000,102.000000,136.000000
P1,137.000000,4.014599,2.910391,1.000000,2.000000,3.000000,4.000000,12.000000
P2,137.000000,4.408759,1.514900,1.000000,4.000000,5.000000,5.000000,7.500000
P3,137.000000,4.317518,1.032337,0.000000,4.000000,4.000000,5.000000,7.500000
P4,137.000000,4.372263,1.016462,3.000000,4.000000,4.000000,5.000000,7.500000
P5,137.000000,2.007299,1.209620,1.000000,1.000000,2.000000,2.000000,8.000000
P6,137.000000,3.357664,2.134235,1.000000,2.000000,3.000000,4.000000,10.000000
P7,137.000000,5.423358,2.296809,1.000000,5.000000,5.000000,5.000000,10.000000
P8,137.000000,5.153285,1.858567,1.000000,4.000000,5.000000,5.000000,10.000000
P9,137.000000,5.445255,1.834793,4.000000,4.000000,5.000000,5.000000,10.000000


In [338]:
x = dtrain.drop(columns = 'revenue', axis = 1)
y = dtrain['revenue']

In [339]:
x

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,2.0,3.0,5,3,4,5,5,4,3,4
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,3.0,0,0,0,0,0,0,0,0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,1.0,3.0,0,0,0,0,0,0,0,0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,2.5,7.5,25,12,10,6,18,12,12,6
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,1.0,3.0,5,1,3,2,3,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,132,06/25/2008,Trabzon,Other,FC,2,3.0,3.0,5.0,4,...,2.0,3.0,0,0,0,0,0,0,0,0
133,133,10/12/2006,İzmir,Big Cities,FC,4,5.0,4.0,4.0,2,...,3.0,3.0,0,0,0,0,0,0,0,0
134,134,07/08/2006,Kayseri,Other,FC,3,4.0,4.0,4.0,2,...,2.0,3.0,0,0,0,0,0,0,0,0
135,135,10/29/2010,İstanbul,Big Cities,FC,4,5.0,4.0,5.0,2,...,3.0,3.0,0,0,0,0,0,0,0,0


In [340]:
y

0      5653753.0
1      6923131.0
2      2055379.0
3      2675511.0
4      4316715.0
         ...    
132    5787594.0
133    9262754.0
134    2544857.0
135    7217634.0
136    6363241.0
Name: revenue, Length: 137, dtype: float64

In [341]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 31)

In [342]:
categ = [x_train.columns.get_loc(i) for i in x_train.dtypes[x_train.dtypes == 'object'].index]

In [343]:
cat

In [344]:
params = {
    'iterations': 3000,
    'eval_metric': 'RMSE',
    'depth': 16,
    'l2_leaf_reg':3,
    'verbose': 100,
    'learning_rate':0.03,
    'od_type':'Iter',
    'od_wait':100
}
cat = CatBoostRegressor(**params)
cat.fit(x_train,y_train,cat_features=categ,eval_set = (x_test,y_test),plot = True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 2730974.1131176	test: 1658513.7362478	best: 1658513.7362478 (0)	total: 66.3ms	remaining: 3m 18s
100:	learn: 1854068.7624199	test: 1639543.2949050	best: 1592008.5238748 (35)	total: 11.4s	remaining: 5m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 1592008.524
bestIteration = 35

Shrink model to first 36 iterations.


In [379]:
pred = cat.predict(dtest)

In [381]:
pred

array([4338261.77089112, 4281538.18538065, 4384244.3760056 , ...,
       4918999.76890776, 4670883.27258722, 4608023.49081253])